<h1 align='center'>Prompts & Agents</h1>

<h2 align='center'>How to incorporate prompting into your Python scripts <br>and expand their functionality through agents</h2>

<h2 align='center'>Laura Funderburk</h2>

<h2 align='center'>PyData Vancouver</h2>

<h1 align='center'>Talk at a glance</h1>

<h2 align='center'>Part I: Prompting (25 minutes)</h2>


1. LLMs use cases and tasks
2. The Generative AI project lifecycle 
3. Choosing the right LLM for the desired task
4. Key elements of prompting
5. Prompting techniques: zero-shot inference, one-shot inference, few-shot inference vs Chain of Thought 
6. Prompting private LLMs (OpenAI API): `ChatCompletion`
7. Roles in the `ChatCompletion` end point (system, user, assistant) and context within prompting
8. Prompting open source LLMs through HuggingFace (google/flan-t5-base)


Q&A about prompting (5 minutes)

<h1 align='center'>Talk at a glance</h1>

<h2 align='center'>Part II: Incorporating agents (25 minutes)</h2>


1. (Some) Applications of agents
2. Introduction to Haystack
3. Introduction to LangChain
4. Which one to pick
5. Techniques to combine prompting and agents

Q&A about agents (5 minutes)

<h1 align='center'>LLMs use cases and tasks</h1>


<h1 align='center'>The generative AI project lifecycle</h1>

<p></p>

<center>
  <img src="diagrams/genai_project_lifecycle.jpg" width="1500px"/>

</center>

<h1 align='center'>Focus of this talk</h1>

<p></p>
<center>
  <img src="diagrams/genai_project_lifecycle_focus.jpg" width="1500px"/>

</center>

<h1 align='center'>Choosing the right LLM for the desired task</h1>




<h1 align='center'>Key elements of prompting</h1>



<h1 align='center'>Prompting techniques</h1>

zero-shot inference, one-shot inference, few-shot inference vs Chain of Thought 



<h1 align='center'>Prompting private LLMs (OpenAI API)</h1>


` ChatCompletion`

<h1 align='center'>Roles in prompting the ChatCompletion endpoint</h1>



<h1 align='center'>Prompting open source LLMs through HuggingFace (google/flan-t5-base)</h1>


In [54]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig
import transformers
import torch
from dotenv import load_dotenv
import os
import openai

load_dotenv(".env")

open_ai_key = os.environ.get("openai-key")

 

In [21]:
import banking  # noqa: E402

_ = banking.BankingData("https://tinyurl.com/jb-bank", "bank")
_.extract_to_csv()

In [108]:
class Prompter:
    def __init__(self, api_key, gpt_model, temperature=0.2):
        if not api_key:
            raise Exception("Please provide the OpenAI API key")

        self.api_key  = api_key
        self.gpt_model = gpt_model
        self.temperature = temperature
    
    def prompt_model_return(self, messages: list):
        openai.api_key = self.api_key
        response = openai.ChatCompletion.create(model=self.gpt_model, 
                                                messages=messages,
                                                temperature=self.temperature)
        return response["choices"][0]["message"]["content"]
    
    def natural_language_to_sql(self, db_name:str, schema:str, natural_question:str):

        system_content = f"You are a data analyst, and you specialize in solving business questions with SQL.\
                        You are given a natural language question, and your role is to translate the question\
                        into a query that can be executed against a database. \
                        Ensure your queries are written in a single line, with no special characters"
        user_content = f"Please generate a SQL query for data with in a database named {db_name}\
                        along with a schema {schema} for the question {natural_question}"

        full_prompts = [
                                {"role" : "system", "content" : system_content},
                                {"role" : "user", "content" : user_content},
                                ]
        
        result = self.prompt_model_return(full_prompts)

        return result

In [101]:
class OpenSourcePrompter:
    def __init__(self, model_name):
        self.model_name = model_name
        
    def call_model(self, full_prompt):
        model = AutoModelForSeq2SeqLM.from_pretrained(self.model_name)
        tokenizer = AutoTokenizer.from_pretrained(self.model_name, use_fast=True)
        
        inputs = tokenizer(full_prompt, return_tensors='pt')
        output = tokenizer.decode(
            model.generate(
                inputs["input_ids"], 
                max_new_tokens=50,
            )[0], 
            skip_special_tokens=True
        )
        
        return output
        
    def natural_language_to_sql(self, db_name:str, schema:list, natural_question:str):

        prompt = f"Given the natural language question {natural_question}\
                        the database table {db_name}, \
                        and the table schema {', '.join(schema)}\
                        generate a SQL query that answers the question {natural_question}"

    
        result = self.call_model(prompt)

        return result

In [91]:
# Loading in SQL extension
%reload_ext sql
# Initiating a DuckDB database named 'bank.duck.db' to run our SQL queries on
%sql duckdb:///bank.duck.db

In [92]:
%%sql
CREATE OR REPLACE TABLE bank AS
FROM read_csv_auto('bank_cleaned.csv', header=True, sep=',')

Running query in 'duckdb:///bank.duck.db'

Count
4521


In [93]:
columns = %sql PRAGMA table_info('bank');

# Extract column names
column_names = [row[1] for row in columns]

Running query in 'duckdb:///bank.duck.db'

In [109]:
pm  = Prompter(open_ai_key, "gpt-4")

pm.natural_language_to_sql("bank", column_names, "How many records are there?")

'SELECT COUNT(*) FROM bank;'

In [106]:
opm = OpenSourcePrompter("google/flan-t5-base")

In [107]:
opm.natural_language_to_sql("bank", column_names, "How many records are there?")

'Count the number of records in the database table bank: age, job, marital, education, default, balance, housing, loan, contact, day, month, duration, campaign, pdays, previous, poutcome'

In [28]:
%sqlcmd explore --table bank

In [8]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

Found cached dataset csv (/Users/macpro/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-c8fac5d84cd35861/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [67]:
model_name='google/flan-t5-base'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [11]:
from haystack.nodes import PromptNode

# Initalize the node passing the model:
prompt_node = PromptNode(model_name_or_path="google/flan-t5-base")


In [12]:
# Go ahead and ask a question:
prompt_node("What is the best city in Europe to live in?")


['san marino']

In [ ]:
{"system"}

In [17]:
pn = PromptNode("gpt-4", api_key=open_ai_key)


In [20]:
prompt = "What is the best city in Europe to live in?"
pn(prompt, max_tokens=100)

['Determining the "best" city in Europe to live in is subjective as it highly depends on personal preferences such as lifestyle, language, cost of living, safety, climate, job opportunities, etc. However, according to various surveys and rankings, cities like Copenhagen, Denmark; Vienna, Austria; Zurich, Switzerland; Munich, Germany; and Amsterdam, Netherlands consistently rank high in terms of quality of life.']